In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import cv2
import os 
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image
import random
import matplotlib.image as mpimg
from imutils import paths
import sys
import operator

In [3]:
def get_paths_of_lines_and_pages_v3(book_path):
  book = sorted(os.listdir(book_path), key = lambda x: int(x))
  sheet_paths = []
  subpages = []
  lines = []
  for page in book:
    sheet_paths.append(book_path+"/"+page)

  for path in sheet_paths:
    page_parts = sorted(os.listdir(path), key = lambda x: int(x))
    page_parts_paths = []
    for part in page_parts:
      page_parts_paths.append(path+"/"+part)
    subpages.append(page_parts_paths)

  for page in subpages:
    sheet = []
    for part in page:
      page_lines = sorted(os.listdir(part), key = lambda x: int(x.split(".")[0]))
      lines_path = []
      for line in page_lines:
        lines_path.append(part+"/"+line)
      sheet.append(lines_path)
    lines.append(sheet)

  return {
      "sheet_paths": sheet_paths,
      "subpage_paths": subpages,
      "line_paths": lines
  }

In [4]:
pages_path = "/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17"
book_paths = get_paths_of_lines_and_pages_v3(pages_path)

In [5]:
book_paths["sheet_paths"][:12]

['/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/3',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/4',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/5',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/6',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/7',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/8',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/9',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/10',
 '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/11',
 '/content/drive/MyDrive/Line Extracti

In [6]:
book_paths["subpage_paths"][:2]

[['/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/202'],
 ['/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/3/301',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/3/302']]

In [7]:
book_paths["line_paths"][0]

[['/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201001.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201002.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201003.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201004.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201005.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201006.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201007.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/201008.png',
  '/content/drive/MyDrive/Line Extraction/Images/Image Lines/menakıb kazeruniOMERFAZILAKOZ17/2/201/20100

In [8]:
def fill_page_nums_with_zero(book_paths):
  for page in book_paths["line_paths"]:
    for part in page:
      for line in part:
        line_num = line.split("/")[-1].split(".")[0]
        parent_path = "/".join(line.split("/")[:-1])
        if len(line_num) == 6:
          new_num = "000" + line_num
          path = parent_path + "/" + new_num + ".png"
          os.rename(line, path)
        if len(line_num) == 7:
          new_num = "00" + line_num
          path = parent_path + "/" + new_num + ".png"
          os.rename(line, path)
        if len(line_num) == 8:
          new_num = "0" + line_num
          path = parent_path + "/" + new_num + ".png"
          os.rename(line, path)
          

In [9]:
fill_page_nums_with_zero(book_paths)

In [10]:
x = "000202032"
y = list(x)
y[4], y[6] = "-", "-"
z = "".join(y)
z


'0002-2-32'

In [11]:
def replace_zeros_with_hyphen(book_paths):
  for page in book_paths["line_paths"]:
    for part in page:
      for line in part:
        line_num = line.split("/")[-1].split(".")[0]
        parent_path = "/".join(line.split("/")[:-1])
        line_num_list = list(line_num)
        line_num_list[4], line_num_list[6] = "-", "-"
        line_num_with_hyphen = "".join(line_num_list)
        new_line = parent_path + "/" + line_num_with_hyphen + ".png"
        os.rename(line, new_line)

In [12]:
book_paths = get_paths_of_lines_and_pages_v3(pages_path)

In [13]:
replace_zeros_with_hyphen(book_paths)